<a href="https://colab.research.google.com/github/RVC-Boss/GPT-SoVITS/blob/main/Colab-WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-SoVITS WebUI

### 1. 須先執行一次

第一次執行有錯誤訊息是正常的

In [ ]:
# -*- coding: utf-8 -*-
# Author: Sunny - Your AI Assistant
# Date:   2025-07-16
# Last Modified by:   Sunny
# Last Modified time: 2025-07-16
# Description:
#   一個整合式的 Colab 啟動器，用於建立並重複使用儲存於 Google Drive 的 Conda 環境。
#   V6.0 (壓縮檔優化版): 使用 tar.gz 壓縮檔作為環境的傳輸媒介，
#   極大提升了 Drive 與 Colab 本地之間的同步速度，徹底解決 I/O 效能瓶頸。

from google.colab import drive
import os
import sys
import shutil

class ColabEnvManager:
    """
    一個管理 Colab 永久性 Conda 環境的類別 (V6.0)。
    """
    def __init__(self, project_dir: str, env_name: str = "env_gsv"):
        # 路徑設定
        self.drive_project_path: str = os.path.join("/content/drive/MyDrive", project_dir)
        self.local_env_path: str = os.path.join("/content", env_name)
        self.env_name = env_name

        # 使用壓縮檔進行傳輸
        self.drive_env_archive_path: str = os.path.join(self.drive_project_path, f"{env_name}.tar.gz")
        self.local_env_archive_path: str = os.path.join("/content", f"{env_name}.tar.gz")

    def _ensure_conda_runtime(self) -> None:
        """確保 Conda 執行環境在當前的 Colab Session 中可用。"""
        if "google.colab" in sys.modules and not os.path.exists("/usr/local/bin/conda"):
            print("⏳ 偵測到新的 Colab Session，正在準備 Conda 執行環境...")
            get_ipython().system('pip install -q condacolab')
            import condacolab
            condacolab.install()
            print("\n✅ Conda 執行環境已就緒。")
            print("⚠️ 為使核心變更生效，Colab 已自動重啟，這屬於正常現象。")
            print("🔥 請您再次執行此儲存格 (Please run this cell again) 以繼續後續步驟！")
            sys.exit()
        try:
            import condacolab
            condacolab.check()
        except ImportError:
            pass

    def mount_drive_and_sync(self) -> None:
        """掛載 Drive 並透過壓縮檔高效地將環境同步到本地 VM。"""
        print("⏳ 正在掛載 Google Drive...")
        if not os.path.isdir("/content/drive/MyDrive"):
            drive.mount("/content/drive")
        print("✅ Google Drive 掛載成功。")
        os.makedirs(self.drive_project_path, exist_ok=True)
        os.chdir(self.drive_project_path)
        print(f"工作目錄已變更至：{self.drive_project_path}")

        # --- 全新的同步邏輯 ---
        if os.path.exists(self.drive_env_archive_path):
            if os.path.exists(self.local_env_path):
                print(f"✅ 本地環境 '{self.local_env_path}' 已存在，跳過同步。")
            else:
                print(f"⚡️ 偵測到 Drive 中的環境壓縮檔，開始高速同步...")
                # 1. 複製單一壓縮檔到本地
                print(f"   - 步驟 1/2: 正在從 Drive 複製 '{os.path.basename(self.drive_env_archive_path)}'...")
                shutil.copy(self.drive_env_archive_path, self.local_env_archive_path)

                # 2. 在本地解壓縮
                print(f"   - 步驟 2/2: 正在於本地解壓縮環境...")
                get_ipython().system(f'tar -xzf {self.local_env_archive_path} -C /content/')

                # 3. 清理本地壓縮檔
                os.remove(self.local_env_archive_path)
                print("✅ 環境高速同步完成！")
        else:
            print(f"⚠️ 在 Drive 中未找到環境壓縮檔，將在首次安裝後為您建立。")

    def setup_and_persist_environment(self) -> None:
        """在本地建立環境，然後將其打包並持久化到 Google Drive。"""
        print("⚠️ 偵測到專案環境不完整，將開始首次安裝...")

        print(f"🛠️ 正在於本地 '{self.local_env_path}' 建立 Conda 環境...")
        get_ipython().system(f'conda create -p "{self.local_env_path}" python=3.10 -y')

        print("📦 正在安裝依賴套件...")
        install_command = f"""
        bash -c "source /usr/local/etc/profile.d/conda.sh && \\
                  conda activate '{self.local_env_path}' && \\
                  pip install ipykernel taibun && \\
                  bash install.sh --device CU126 --source HF --download-uvr5"
        """
        get_ipython().system(install_command)

        print("\n🎉 本地環境安裝成功！")

        # --- 全新的持久化邏輯 ---
        print(f"💾 正在將新環境打包並保存至 Google Drive...")
        # 使用 tar 指令打包本地環境 -C 參數可以避免在壓縮檔中包含 /content/ 的路徑
        get_ipython().system(f'tar -czf {self.drive_env_archive_path} -C /content {self.env_name}')
        print(f"✅ 環境已成功打包並保存至 '{self.drive_env_archive_path}'！")

    def run_main_app(self, main_script: str = "webui.py", share_gradio: bool = False) -> None:
        """在本地的 Conda 環境中執行主應用程式。"""
        print(f"\n🚀 正在從本地路徑啟動應用程式: {main_script}...")
        command_prefix = "export is_share=True &&" if share_gradio else ""
        if share_gradio:
            print("   └── ✨ 已啟用 Gradio 公開分享連結模式。")

        python_executable = os.path.join(self.local_env_path, 'bin', 'python')
        run_command = f"""
        bash -c "source /usr/local/etc/profile.d/conda.sh && \\
                  conda activate '{self.local_env_path}' && \\
                  {command_prefix} {python_executable} {main_script}"
        """
        get_ipython().system(run_command)

    def run(self, main_script: str = "webui.py", share_gradio: bool = False):
        """執行完整流程。"""
        self._ensure_conda_runtime()
        self.mount_drive_and_sync()

        if not os.path.exists(self.local_env_path):
            self.setup_and_persist_environment()

        self.run_main_app(main_script, share_gradio)


# --- 使用方式 (與之前完全相同) ---
project_folder_name = "GPT-SoVITS"
manager = ColabEnvManager(project_dir=project_folder_name)
manager.run(main_script="webui.py", share_gradio=True)

### 2. 看要執行什麼檔案

In [ ]:
# ==============================================================================
# 前提：您已在筆記本的上文成功執行過一次 ColabEnvManager 的初始化與運作
# manager 物件已經存在於當前的 Python session 中
# ==============================================================================

# 假設 manager 物件已由先前的儲存格建立
# assert 'manager' in locals(), "錯誤：請先執行前一個包含 ColabEnvManager 的儲存格"

# --- 執行您的第一個測試腳本 ---
test_script = "generate_list_from_json.py"
print(f"\n▶️ 正準備執行腳本：{test_script}")
# 直接呼叫執行方法，傳入新的腳本名稱
manager.run(main_script=test_script, share_gradio=True)



### 3. 單純掛載專案

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
# 驗證掛載是否成功
print("Google Drive 掛載成功！")
print("您的專案資料夾內容：")
os.system('ls -l /content/drive/MyDrive/GPT-SoVITS')